### 1. junehong/1002_ver_code.ipynb 적용
### 2. 소분류별로 2개 이하인 키워드 제거, 제거된 키워드 모아서 test_df에서도 제거


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd

In [31]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/test.csv')

In [32]:
train_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


In [33]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


In [ ]:
### junehong/1002_ver_code.ipynb

In [34]:
# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df = train_df[condition_train].copy()
filtered_test_df = test_df[condition_test].copy()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 각 행의 '키워드' 값에서 '기사' 단어부터 마지막 단어까지 삭제
filtered_train_df['키워드'] = filtered_train_df['키워드'].apply(remove_keywords)
filtered_test_df['키워드'] = filtered_test_df['키워드'].apply(remove_keywords)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train, '키워드'] = filtered_train_df['키워드']
test_df.loc[condition_test, '키워드'] = filtered_test_df['키워드']

In [35]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|'  # 영어+한글
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[A-Za-z\u4E00-\u9FFF]+|'  # 한글+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자
        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+|'                          # 숫자
        r'[A-Za-z]+'                        # 영어
    )
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            removed_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)


In [36]:
train_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인, 농촌, 테마파크, 단체, 체험객, 체험료, 지원, 체험일, 기준, 용인시통합..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시, 노후, 주택, 에너지, 성능, 개선, 신청, 연장, 용인시청, 용인시, 노..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원, 용인, 고양시, 특례시, 지정, 도시경쟁력, 증가, 경기, 도내, 인구, 수..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시, 스페인, 미국, 국제, 명예, 자문관, 위촉, 역할, 대외, 홍보, 지원,..."


In [37]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수, 별세, 김태수씨, 서울, 광남초등학, 교장, 별세, 김윤정, 이노코리아, ..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규, 확진, 나흘, 세자릿수, 방역당국, 핼러윈, 풍선, 효과, 차단, 총력, 감..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철, 장관, 재정, 분권, 강화, 지방자치, 시대, 마중물, 마련, 장관, 전해..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시, 구인, 장애인, 구직, 만남, 채용, 행사, 노호근, 용인특례시, 장애인,..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체, 경기, 북동부, 지역, 산업단지, 혁신단위, 설정, 전략, 지역, 연계, ..."


In [38]:
def filter_keywords_by_class_count(df, min_count):
    """분류별로 n회 이하로 등장한 키워드를 제거한 DataFrame과 삭제된 키워드를 반환합니다."""
    # 모든 분류에 대해 키워드 집계
    class_keyword_counts = {}

    for label in df['분류'].unique():
        class_df = df[df['분류'] == label]
        keywords = class_df['키워드'].str.cat(sep=',').split(',')
        counts = Counter(keywords)
        class_keyword_counts[label] = counts

    # min_count 이하인 키워드 찾기
    low_count_keywords = {}
    for label, counts in class_keyword_counts.items():
        low_count_keywords[label] = {keyword for keyword, count in counts.items() if count <= min_count}

    # DataFrame의 키워드에서 각 분류별로 low_count_keywords를 제거
    def filter_keywords(row):
        keywords = row['키워드'].split(',')
        filtered_keywords = [keyword for keyword in keywords if keyword not in low_count_keywords[row['분류']]]
        return ','.join(filtered_keywords)

    # 새로운 열에 필터링된 키워드 저장
    df['필터된_키워드'] = df.apply(filter_keywords, axis=1)
    return df, low_count_keywords


In [39]:
train_df, removed_keywords = filter_keywords_by_class_count(train_df, min_count=2)

In [40]:
removed_keywords.keys()

dict_keys(['문화:전시_공연', '지역', '국제', '정치:선거', '경제:금융_재테크', '사회:의료_건강', '정치:행정_자치', '정치:국회_정당', '스포츠:축구', '경제:서비스_쇼핑', '경제:부동산', '사회:교육_시험', '사회:미디어', 'IT_과학:인터넷_SNS', '사회:장애인', '사회:노동_복지', '경제:경제일반', '정치:정치일반', '사회:사회일반', '문화:문화일반', '경제:취업_창업', '사회:사건_사고', '스포츠:올림픽_아시안게임', '사회:환경', '정치:외교', '경제:산업_기업', '스포츠:골프', '경제:유통', 'IT_과학:모바일', '사회:여성', '사회:날씨', '문화:방송_연예', 'IT_과학:IT_과학일반', '경제:반도체', '경제:자원', '문화:종교', '경제:자동차', '경제:무역', 'IT_과학:콘텐츠', '경제:증권_증시', 'IT_과학:과학', '경제:외환', '문화:요리_여행', '정치:청와대', '문화:출판', '문화:미술_건축', '문화:음악', '스포츠:농구_배구', '문화:생활', '정치:북한', '스포츠:야구', '문화:학술_문화재', 'IT_과학:보안', '문화:영화', '스포츠:월드컵', '스포츠:스포츠일반'])

In [41]:
# 제거된 키워드를 합쳐서 test_df에서 필터링
all_removed_keywords = set.union(*removed_keywords.values())

In [42]:
def filter_test_keywords(test_df, removed_keywords):
    """test_df에서 제거된 키워드를 삭제한 DataFrame을 반환합니다."""
    def filter_keywords(row):
        keywords = row['키워드'].split(',')
        filtered_keywords = [keyword for keyword in keywords if keyword not in removed_keywords]
        return ','.join(filtered_keywords)

    # 필터링된 키워드로 업데이트
    test_df['필터된_키워드'] = test_df.apply(filter_keywords, axis=1)
    return test_df

In [43]:
# test_df에서 키워드 필터링
test_df = filter_test_keywords(test_df, all_removed_keywords)

In [44]:
# train_df와 test_df에서 'ID', '제목', '키워드' 열 삭제
train_df = train_df.drop(columns=['ID', '제목', '키워드'])
test_df = test_df.drop(columns=['ID', '제목', '키워드'])

In [45]:
train_df = train_df.rename(columns={'필터된_키워드': '키워드'})
test_df = test_df.rename(columns={'필터된_키워드': '키워드'})

In [46]:
train_df.head()

,분류,키워드
0,문화:전시_공연,"용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,..."
1,지역,"용인, 농촌, 테마파크, 단체, 체험객, 체험료, 지원, 체험일, 기준, 용인시통합..."
2,지역,"용인시, 노후, 주택, 에너지, 성능, 개선, 신청, 연장, 용인시청, 용인시, 노..."
3,지역,"수원, 용인, 고양시, 특례시, 지정, 도시경쟁력, 증가, 경기, 도내, 인구, 수..."
4,국제,"용인시, 스페인, 미국, 국제, 명예, 자문관, 위촉, 역할, 대외, 홍보, 지원,..."


In [47]:
test_df.head()

,키워드
0,"광남초등학, 대표, 도식씨, 경기, 용인시"
1,"세자릿수, 기록, 경기, 지인모임, 경기, 용인시, 경기, 행복해요, 경기, 양원..."
2,"시작, 당정청, 재정분권특별위원회, 경기, 용인시, 특례시지원협의회, 관광개발조합..."
3,"용인시, 용인시"
4,"경기, 경기, 연구작업, 지역연계특성화, 지역연계특성화, 북동부지역, 고양테크노밸..."


In [48]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [49]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [52]:
# 데이터 준비
#train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
#test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [53]:
train_dataset[0]

{'text': '매물, 거래량, 조짐, 집값, 안정, 아시아투데이, 박지숙, 서울, 포함, 매물, 수도, 아파트, 거래량, 매수세, 추세, 주택, 시장, 안정, 주목, 부동산, 시장, 관계자, 전문가들, 중과, 보유세, 세제, 인상, 매물, 적체, 현상, 공급대책, 관망세, 영향, 정부, 인상, 부동산, 세제, 본격적, 시행, 부담, 다주택자들, 매물, 시작, 역대급, 물량, 사전, 청약, 예고, 무주택자들, 매수, 분위기, 해석, 아실, 부동산, 빅데이터, 업체, 아파트실거래가, 매물, 이날, 기준, 서울, 아파트, 매매, 전세, 월세, 지난달, 경기, 이달, 서울, 은평구, 이달, 매물, 서초구, 송파구, 강남구, 강남구, 자치구, 서울, 매물, 상태, 경기도, 안산시, 단원구, 의왕시, 의정부시, 강남, 시세, 분당구, 경기, 성남시, 수지구, 경기, 용인시, 경기, 하남시, 매물, 거래량, 집계, 이날, 기준, 거래량, 절반, 추세, 서울, 거래량, 급감, 중순, 이날, 거래량, 거래량, 비교, 수준, 실거래가, 하락, 추세, 강남, 은마아파트, 거래, 이달, 거래, 송파, 헬리오시티, 거래, 매물, 상태, 관계자, 서초구, 매물, 거래, 보유세, 양도세, 부담, 다주택자들, 연초, 사람, 매매가, 분위기, 본부장, 장재현, 리얼투데이, 보유세, 부담, 매물, 영향, 영향, 다주택자들, 부담, 토지, 부소장, 거래량, 감소, 추세, 매매가, 소득, 극단, 괴리, 구매, 소진, 시장, 상황, 보유세, 인상, 복합적, 작용, 결과, 부소장, 매매거래량, 서울, 아파트, 매매, 거래량, 제외, 비정상적, 가격, 모습, 전형적, 대세, 하락, 초입, 현상, 거래량, 급매물, 본격적, 시작, 설명',
 'input_ids': tensor([   2,    0,   46,    0,   46,    0,   46,    0,   46, 6817,   46, 6802,
           46,    0,   46, 6556,   46,    0,   46,   

In [54]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [55]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 171/171 [00:43<00:00,  3.93it/s]


Validation F1 Score: 0.21


Validating: 100%|██████████| 171/171 [00:43<00:00,  3.93it/s]


Validation F1 Score: 0.31


Validating: 100%|██████████| 171/171 [00:43<00:00,  3.92it/s]


...

Validation F1 Score: 0.50


Validating: 100%|██████████| 171/171 [00:43<00:00,  3.92it/s]


Validation F1 Score: 0.50


Validating: 100%|██████████| 171/171 [00:43<00:00,  3.93it/s]

Validation F1 Score: 0.52


In [56]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [00:43<00:00,  8.45it/s]


In [57]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert.csv", encoding='UTF-8-sig', index=False)

In [58]:
result = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert.csv")
result.head()

,ID,분류
0,TEST_00000,스포츠:월드컵
1,TEST_00001,스포츠:올림픽_아시안게임
2,TEST_00002,지역
3,TEST_00003,스포츠:축구
4,TEST_00004,지역


In [59]:
result['분류'].value_counts()

,count
분류,
지역,10438
스포츠:올림픽_아시안게임,3712
스포츠:월드컵,2463
경제:경제일반,1674
정치:국회_정당,1448
스포츠:축구,1219
사회:여성,1112
경제:서비스_쇼핑,316
문화:출판,283
